In [2]:
import pymongo
import copy
import cpi

In [3]:
from tqdm import tqdm

In [4]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["imdb"]
imdb = db['movies']

In [5]:
def getMovie(imdb_id):
    for x in imdb.find({'imdb_id': imdb_id}):
        return x
    return None

In [6]:
test_movie = getMovie('tt0284020')
test_movie

{'_id': ObjectId('60a66511ef235d48e4896633'),
 '@context': 'http://schema.org',
 '@type': 'Movie',
 'url': '/title/tt0284020/',
 'name': 'The Cure for Insomnia',
 'image': 'https://m.media-amazon.com/images/M/MV5BNjk3NWU1MzMtZWZjZS00NmY5LTkxZTQtNGRjZTdmOTM1MDUyXkEyXkFqcGdeQXVyMjM5NDM1MTE@._V1_.jpg',
 'genre': ['Documentary', 'Music'],
 'contentRating': 'Unrated',
 'actor': [{'@type': 'Person',
   'url': '/name/nm1523971/',
   'name': 'Cosmic Lightning'},
  {'@type': 'Person', 'url': '/name/nm1007137/', 'name': 'L.D. Groban'},
  {'@type': 'Person', 'url': '/name/nm1523603/', 'name': 'J.T.4'}],
 'director': {'@type': 'Person',
  'url': '/name/nm1267224/',
  'name': 'John Henry Timmis IV'},
 'creator': [{'@type': 'Person',
   'url': '/name/nm1007137/',
   'name': 'L.D. Groban'},
  {'@type': 'Organization', 'url': '/company/co0084142/'}],
 'description': 'The Cure for Insomnia is a movie starring Cosmic Lightning, L.D. Groban, and J.T.4. Not really following any standard plot structure, th

In [7]:
months =    ['January', 'February', 'March', 'April', 'May', 'June',
            'July', 'August', 'September', 'October', 'November', 'December']
years = []
for year in range(1967, 2021):
    years.append(str(year))

days = []
for day in range(1, 32):
    days.append(str(day))

In [8]:
def getDate(date_str):
    date_str = date_str.split(" ")
    day = None
    month = None
    year = None

    for val in date_str:
        if(day == None and val in days):
            day = val
        if(month == None and val in months):
            month = val
        if(year == None and val in years):
            year = val
    
    return day, month, year

In [9]:
def getUSD(txt):
    # converter = {
    #     '$'  : 1, 'USD': 1, 'EUR': 1.22, 'AUD': 0.78,
    #     'GBP': 1.41, 'NOK': 0.12, 'DKK': 0.16, 'JPY': 0.0092,
    #     'CAD': 0.83, 'INR': 0.012, 'SEK': 0.12, 'DEM': 0.62,
    #     'FRF': 0.185, 'RUR': 0.014, 'CNY': 0.16, 'BEF': 1/33.5,
    #     'MXN': 0.050, 'CZK': 0.048, 'ZAR': 0.071, 'HKD': 0.13,
    #     'BRL': 0.19, 'IDR': 0.000070, 'ITL': 0.000626628, 'ARS': 0.011,
    #     'UAH': 0.036, 'NZD': 0.73, 'TRL': 0.12, 'FIM': 0.204585,
    #     'HUF': 0.0034, 'DOP': 0.018, 'KRW': 0.00089, 'ESP': 1.22, 
    #     'EGP': 0.064, 'PLN': 0.27, 'SGD': 0.75, 'IQD': 0.00068, 
    #     'IEP': 0.647361, 'CHF': 1.11, 'ISK': 0.0081, 'TWD': 0.036,
    #     'THB': 0.032, 'MYR': 0.24, 'PYG': 0.00015, 'NGN': 0.0026, 
    #     'ILS': 0.30, 'IRR': 0.000024, 'NLG': 0.039, 'BDT': 0.012, 
    #     'AMD': 0.0019, 'MVR': 0.065, 'SAR': 0.27, 'RON': 0.25,
    #     'ATS': 0.0883949, 'PHP': 0.021, 'ROL': 0.25
    # }
    converter = {
        '$'  : 1, 'USD': 1
    }
    txt = txt.replace(",","")
    mul = 1
    for key in converter:
        if key in txt:
            txt = txt.replace(key, "")
            mul = converter[key]
            break
    money = int(txt)
    return money

In [10]:
def adjustInflation(amount, year):
    try:
        year = int(year)
        return int(cpi.inflate(amount, year))
    except:
        return amount

In [11]:
def cleanMovie(movie, print_log = False):
    # RELEASE DATE
    movie = copy.deepcopy(movie)
    try:
        date_str = movie['details']['Release Date'][0]
        day, month, year = getDate(date_str)
        if(month == None or year == None):
            if(print_log == True):
                print("{} Problem loading month {} or year {} info".format(movie['imdb_id'],month, year))
            return None
        movie['cleaned_month'] = month
        movie['cleaned_year'] = year

    except:
        if(print_log == True):
            if('Release Date' not in movie['details']):
                print("{} This movie does not have <Release Date> attribute".format(movie['imdb_id']))
            else:
                print(movie['imdb_id'], "Problem parsing release date attribut ==> ", movie['details']['Release Date'])
        return None


    # CONTENT RATING
    if('contentRating' not in movie):
        if(print_log == True):
            print(movie['imdb_id'], "This movie does not have <contentRating> attribute")
        return None
    movie['cleaned_contentRating'] = movie['contentRating']


    # RUNTIME
    if('Runtime' not in movie['details']):
        if(print_log == True):
            print(movie['imdb_id'], "This movie does not have <Runtime> attribute")
        return None
    runtime = movie['details']['Runtime'][0]
    runtime = int(runtime.split()[0].replace(',',''))
    movie['cleaned_Runtime_min'] = runtime


    # BUDGET
    if('Budget' not in movie['details']):
        if(print_log == True):
            print(movie['imdb_id'], "This movie does not have <Budget> attribute")
        return None
    budget = movie['details']['Budget'][0]
    try:
        budget = getUSD(budget)
    except:
        if(print_log == True):
            print(movie['imdb_id'], "This movie does not have <Budget> attribute in a convertable format")
        return None
    # movie['cleaned_budget'] = budget
    movie['cleaned_Budget'] = adjustInflation(budget, movie['cleaned_year'])


    # REVENUE
    if('Cumulative Worldwide Gross' not in movie['details']):
        if(print_log == True):
            print(movie['imdb_id'], "This movie does not have <Cumulative Worldwide Gross> attribute")
        return None
    revenue = movie['details']['Cumulative Worldwide Gross'][0]
    try:
        revenue = getUSD(revenue)
    except:
        if(print_log == True):
            print(movie['imdb_id'], "This movie does not have <Revenue> attribute in a convertable format")
        return None
    movie['cleaned_Revenue'] = adjustInflation(revenue, movie['cleaned_year'])

    return movie

In [12]:
adjustInflation(585796247, 2008)

704172781

In [13]:
cleanMovie(test_movie, print_log=True)

tt0284020 This movie does not have <Budget> attribute


In [14]:
clean_failed = 0
clean_success = 0
limit = imdb.count_documents({})
step_size = 1000
cnt = 0

movie_list = []
for movie in imdb.find().limit(limit):
    try:
        cleaned_movie = cleanMovie(movie)
        if(cleaned_movie == None):
            clean_failed += 1
        else:
            clean_success += 1
            movie_list.append(cleaned_movie)
    except:
        print(movie['imdb_id'])
        break
    cnt += 1
    if(cnt % step_size == 0):
        print("processed {} out of {} --- success {} , fail {}".format(cnt, limit, clean_success, clean_failed))

print('processed {} -- failed {}'.format(clean_success, clean_failed))

processed 1000 out of 205951 --- success 699 , fail 301
processed 2000 out of 205951 --- success 1438 , fail 562
processed 3000 out of 205951 --- success 2107 , fail 893
processed 4000 out of 205951 --- success 2714 , fail 1286
processed 5000 out of 205951 --- success 3262 , fail 1738
processed 6000 out of 205951 --- success 3736 , fail 2264
processed 7000 out of 205951 --- success 4141 , fail 2859
processed 8000 out of 205951 --- success 4489 , fail 3511
processed 9000 out of 205951 --- success 4806 , fail 4194
processed 10000 out of 205951 --- success 5087 , fail 4913
processed 11000 out of 205951 --- success 5312 , fail 5688
processed 12000 out of 205951 --- success 5500 , fail 6500
processed 13000 out of 205951 --- success 5687 , fail 7313
processed 14000 out of 205951 --- success 5864 , fail 8136
processed 15000 out of 205951 --- success 5994 , fail 9006
processed 16000 out of 205951 --- success 6128 , fail 9872
processed 17000 out of 205951 --- success 6246 , fail 10754
processed

In [15]:
imdb.count_documents({})

205951

In [16]:
len(movie_list)

8181

In [17]:
movie_list[20]

{'_id': ObjectId('60a6633eef235d48e488a2d1'),
 '@context': 'http://schema.org',
 '@type': 'Movie',
 'url': '/title/tt6723592/',
 'name': 'Tenet',
 'image': 'https://m.media-amazon.com/images/M/MV5BYzg0NGM2NjAtNmIxOC00MDJmLTg5ZmYtYzM0MTE4NWE2NzlhXkEyXkFqcGdeQXVyMTA4NjE0NjEy._V1_.jpg',
 'genre': ['Action', 'Sci-Fi', 'Thriller'],
 'contentRating': 'PG-13',
 'actor': [{'@type': 'Person',
   'url': '/name/nm0913475/',
   'name': 'John David Washington'},
  {'@type': 'Person', 'url': '/name/nm1500155/', 'name': 'Robert Pattinson'},
  {'@type': 'Person', 'url': '/name/nm4456120/', 'name': 'Elizabeth Debicki'},
  {'@type': 'Person', 'url': '/name/nm1798449/', 'name': 'Juhan Ulfsak'}],
 'director': {'@type': 'Person',
  'url': '/name/nm0634240/',
  'name': 'Christopher Nolan'},
 'creator': [{'@type': 'Person',
   'url': '/name/nm0634240/',
   'name': 'Christopher Nolan'},
  {'@type': 'Organization', 'url': '/company/co0002663/'},
  {'@type': 'Organization', 'url': '/company/co0147954/'}],
 'des

In [18]:
imdb_cleaned = db['imdb_cleaned']
# imdb_cleaned.drop()

In [19]:
for i in tqdm(range(0, len(movie_list))):
    imdb_cleaned.insert_one(movie_list[i])

100%|██████████| 8181/8181 [00:12<00:00, 641.47it/s]


In [20]:
imdb_cleaned.count_documents({})

8181